# Análise Exploratória de Dados (EDA) - Dataset de Sepsis

## Objetivo da Análise

* Compreender a estrutura e qualidade dos dados
* Analisar padrões de valores faltantes
* Identificar relações entre variáveis e a ocorrência de sepsis
* Gerar insights para construção de modelos preditivos

## Sobre o Dataset

* **Problema**: Detecção precoce de sepsis em pacientes de UTI
* **Tipo**: Classificação binária (Sepsis vs Não-Sepsis)
* **Características**: Dados de sinais vitais, exames laboratoriais e informações demográficas


## 1. Importação das Bibliotecas

Primeiro, vamos importar todas as bibliotecas necessárias para nossa análise exploratória:

In [ ]:
# Importação das bibliotecas essenciais para análise exploratória
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configurações para otimizar as visualizações
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10


print("Bibliotecas importadas com sucesso")

## 2. Carregamento e Estrutura do Dataset

Nesta seção, carregamos o dataset de treino e analisamos suas características básicas para compreender a estrutura dos dados e identificar possíveis problemas de qualidade.

In [ ]:
# Carregamento do dataset de treino
train_df = pd.read_csv('dataset_sepsis_train.csv')

# Análise da estrutura básica do dataset
print("INFORMAÇÕES GERAIS DO DATASET")
print("=" * 50)
print(f"Dimensões do dataset (shape): {train_df.shape}")
print(f"Número de amostras: {train_df.shape[0]:,}")
print(f"Número de features: {train_df.shape[1]}")
print(f"Tamanho em memória: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nINFORMAÇÕES DETALHADAS DO DATASET")
print("=" * 50)
# Usar info() para mostrar tipos de dados e valores não-nulos
train_df.info()

print(f"\nESTATÍSTICAS DESCRITIVAS (TRANSPOSTAS)")
print("=" * 50)
# Mostrar estatísticas descritivas transpostas para melhor visualização
display(train_df.describe().T)

# Visualizar as primeiras linhas
print(f"\nPRIMEIRAS 5 LINHAS DO DATASET")
print("=" * 50)
display(train_df.head())

### 2.1 Análise de Valores Faltantes

A análise sistemática de valores faltantes é essencial para identificar padrões de missings e definir estratégias de tratamento adequadas. Esta análise nos permite entender se os dados faltam de forma aleatória ou seguem algum padrão específico.

In [ ]:
# Análise quantitativa dos valores faltantes
missing_count = train_df.isnull().sum()
missing_pct = (missing_count / len(train_df)) * 100

# Criar DataFrame para análise organizada
missing_df = pd.DataFrame({
    'Coluna': missing_count.index,
    'Valores_Faltantes': missing_count.values,
    'Porcentagem': missing_pct.values
}).sort_values('Porcentagem', ascending=False)

# Estatísticas gerais
total_samples = len(train_df)
mean_missing = missing_pct.mean()
vars_with_missing = (missing_count > 0).sum()
total_vars = len(missing_count)

print("ANÁLISE DE VALORES FALTANTES")
print("=" * 50)
print(f"Total de amostras: {total_samples:,}")
print(f"Total de variáveis: {total_vars}")
print(f"Variáveis com missing values: {vars_with_missing}")
print(f"Percentual médio de missing values: {mean_missing:.3f}%")

print(f"\nTOP 10 VARIÁVEIS COM MAIS MISSING VALUES:")
top_10_missing = missing_df.head(10)
display(top_10_missing)

# Categorização por nível de missing
very_high = missing_df[missing_df['Porcentagem'] > 50]
high_missing = missing_df[(missing_df['Porcentagem'] > 20) & (missing_df['Porcentagem'] <= 50)]
medium_missing = missing_df[(missing_df['Porcentagem'] > 5) & (missing_df['Porcentagem'] <= 20)]
low_missing = missing_df[(missing_df['Porcentagem'] > 0) & (missing_df['Porcentagem'] <= 5)]

print(f"\nCATEGORIZAÇÃO POR NÍVEL DE MISSING:")
print(f"Muito Alto (>50%): {len(very_high)} variáveis")
print(f"Alto (20-50%): {len(high_missing)} variáveis")
print(f"Médio (5-20%): {len(medium_missing)} variáveis")
print(f"Baixo (0-5%): {len(low_missing)} variáveis")
print(f"Sem missing: {len(missing_df[missing_df['Porcentagem'] == 0])} variáveis")

### 2.2 Visualização dos Padrões de Missing Values

A visualização dos padrões de valores faltantes nos ajuda a identificar se existe alguma estrutura nos dados ausentes e a definir estratégias de tratamento mais eficazes.

In [ ]:
# Filtrar apenas variáveis com missing values para visualização
missing_with_nans = missing_df[missing_df['Porcentagem'] > 0]

# Configurar subplots - 2 gráficos principais
fig, axes = plt.subplots(1, 2, figsize=(20, 12))

# 1. Gráfico de barras horizontais - TODAS as colunas com missing values
# Preparar dados para o seaborn barplot
missing_sorted = missing_with_nans.sort_values('Porcentagem', ascending=True)  # Crescente para barras horizontais

# Criar gráfico de barras horizontais usando seaborn
sns.barplot(
    x=missing_sorted['Porcentagem'],
    y=missing_sorted['Coluna'],
    orient='h',
    palette='rocket_r',  # Paleta de cores
    ax=axes[0]
)

# Configurar títulos e labels
axes[0].set_title(f'Percentual de Valores Nulos (NaNs) por Coluna\n({len(missing_sorted)} variáveis com dados ausentes)', 
                    fontsize=14, pad=20)
axes[0].set_xlabel('Percentual (%)', fontsize=12)
axes[0].set_ylabel('Colunas do Dataset', fontsize=12)

# Adicionar valores nas barras
for i, (idx, row) in enumerate(missing_sorted.iterrows()):
    pct = row['Porcentagem']
    axes[0].text(pct + 1, i, f'{pct:.1f}%', 
                ha='left', va='center', fontsize=9, fontweight='bold')

# Adicionar linhas de referência
axes[0].axvline(20, color='orange', linestyle='--', alpha=0.8, label='20% (Baixo)')
axes[0].axvline(80, color='red', linestyle='--', alpha=0.8, label='80% (Alto)')
axes[0].legend(loc='upper right')

# Configurar grid e limites
axes[0].grid(axis='x', alpha=0.3)
axes[0].set_xlim(0, 105)

# 2. Histograma da distribuição de missing values (mais segmentado)
# Criar bins de 10 em 10
bins = range(0, 101, 10) 

axes[1].hist(missing_with_nans['Porcentagem'], bins=bins, alpha=0.7, 
            color='#34495e', edgecolor='black', rwidth=0.8)

axes[1].set_xlabel('Percentual de Missing Values (%)', fontsize=12)
axes[1].set_ylabel('Número de Variáveis', fontsize=12)
axes[1].set_title('Distribuição de Missing Values por Faixa', fontsize=14, pad=20)

# Configurar eixo X 
axes[1].set_xticks(bins)
axes[1].set_xlim(0, 100)

# Adicionar linha vertical para a média
mean_missing = missing_with_nans['Porcentagem'].mean()
axes[1].axvline(mean_missing, color='red', linestyle='--', linewidth=2,
                label=f'Média: {mean_missing:.1f}%')
axes[1].legend()

# Configurar grid
axes[1].grid(axis='y', alpha=0.3)

# Adicionar anotações no histograma
counts, bins_edges = np.histogram(missing_with_nans['Porcentagem'], bins=bins)
for i, count in enumerate(counts):
    if count > 0:
        bin_center = (bins_edges[i] + bins_edges[i+1]) / 2
        axes[1].text(bin_center, count + max(counts)*0.01, str(count), 
                    ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

# Análise estatística dos missing values (mais detalhada)
print("ANÁLISE DOS MISSING VALUES")
print("=" * 60)

# Categorização mais detalhada
very_low = missing_with_nans[missing_with_nans['Porcentagem'] < 10]
low_missing = missing_with_nans[(missing_with_nans['Porcentagem'] >= 10) & 
                                (missing_with_nans['Porcentagem'] < 20)]
medium_missing = missing_with_nans[(missing_with_nans['Porcentagem'] >= 20) & 
                                    (missing_with_nans['Porcentagem'] < 50)]
high_missing = missing_with_nans[(missing_with_nans['Porcentagem'] >= 50) & 
                                (missing_with_nans['Porcentagem'] < 80)]
very_high = missing_with_nans[missing_with_nans['Porcentagem'] >= 80]

print("Categorização detalhada por nível de missing values:")
print(f"  • Muito baixo (<10%): {len(very_low)} variáveis")
print(f"  • Baixo (10-20%): {len(low_missing)} variáveis")
print(f"  • Médio (20-50%): {len(medium_missing)} variáveis")
print(f"  • Alto (50-80%): {len(high_missing)} variáveis")
print(f"  • Muito alto (≥80%): {len(very_high)} variáveis")

print(f"\nEstatísticas descritivas dos missing values:")
print(f"  • Média: {missing_with_nans['Porcentagem'].mean():.1f}%")
print(f"  • Mediana: {missing_with_nans['Porcentagem'].median():.1f}%")
print(f"  • Desvio padrão: {missing_with_nans['Porcentagem'].std():.1f}%")
print(f"  • Mínimo: {missing_with_nans['Porcentagem'].min():.1f}%")
print(f"  • Máximo: {missing_with_nans['Porcentagem'].max():.1f}%")

print(f"\nTop 5 variáveis com mais missing values:")
top_5_missing = missing_with_nans.head(5)
for idx, (_, row) in enumerate(top_5_missing.iterrows(), 1):
    print(f"  {idx}. {row['Coluna']}: {row['Porcentagem']:.1f}% ({row['Valores_Faltantes']:,} valores)")

Estratégias de tratamento possíveis por faixa de valores faltantes 

- Muito baixo/Baixo (<20%): Imputação simples (mediana/moda)
- Médio (20-50%): Imputação específica por domínio clínico ()
- Alto (50-80%): Considerar remoção ou imputação avançada
- Muito alto (≥80%): Forte candidato à remoção. Chance de tratamento se estiver associado a muitas instâncias de SepsisLabel=1


#### 2.2.1 Análise Visual de Outliers e Inconsistências

Baseando-se nas estatísticas descritivas, vamos identificar e visualizar variáveis com potenciais problemas de qualidade dos dados, incluindo outliers extremos e valores fisiologicamente inconsistentes.

In [ ]:
# Análise de outliers e inconsistências baseada nas estatísticas descritivas
# Obter estatísticas descritivas apenas para variáveis numéricas (excluindo SepsisLabel)
numerical_vars = train_df.select_dtypes(include=[np.number]).columns
numerical_vars = numerical_vars.drop('SepsisLabel')  # Remover a variável target


# Criar boxplot para visualizar distribuições e outliers
# Como temos muitas variáveis, vamos fazer um gráfico grande com subplots
n_vars = len(numerical_vars)
n_cols = 4  # 4 gráficos por linha
n_rows = (n_vars + n_cols - 1) // n_cols  # Arredondar para cima

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
axes = axes.flatten() if n_vars > 1 else [axes]

for i, var in enumerate(numerical_vars):
    # Criar boxplot para cada variável
    data = train_df[var].dropna()
    
    if len(data) > 0:
        axes[i].boxplot(data, patch_artist=True, 
                       boxprops=dict(facecolor='lightblue', alpha=0.7),
                       medianprops=dict(color='red', linewidth=2),
                       flierprops=dict(marker='o', markerfacecolor='red', markersize=2, alpha=0.5))
        
        axes[i].set_title(f'{var}\n(n={len(data):,})', fontsize=10, pad=10)
        axes[i].set_ylabel('Valores')
        axes[i].grid(True, alpha=0.3)
        
        # Adicionar estatísticas básicas no gráfico
        stats_text = f'Min: {data.min():.2f}\nMédia: {data.mean():.2f}\nMax: {data.max():.2f}'
        axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes, 
                    fontsize=8, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    else:
        axes[i].text(0.5, 0.5, 'Sem dados\nválidos', 
                    transform=axes[i].transAxes, ha='center', va='center')
        axes[i].set_title(var, fontsize=10)

# Remover subplots extras se houver
for i in range(n_vars, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### 2.3 Estrutura Temporal dos Dados

Um aspecto fundamental para compreender este dataset é que **cada linha não representa um paciente único**, mas sim um **momento específico no tempo** para diferentes pacientes. O dataset é uma coleção de séries temporais clínicas onde múltiplas observações são registradas para cada paciente ao longo do tempo.

In [ ]:
print("ANÁLISE DA ESTRUTURA TEMPORAL DOS DADOS")
print("=" * 60)

patient_id_cols = [col for col in train_df.columns if 'patient' in col.lower() or 'id' in col.lower()]
print(f"Colunas que podem ser ID do paciente: {patient_id_cols}")

print(f"\nDIMENSÕES DO DATASET:")
print(f"   Total de observações (linhas): {len(train_df):,}")
print(f"   Total de variáveis (colunas): {len(train_df.columns)}")

print(f"\nVARIÁVEIS TEMPORAIS IDENTIFICADAS:")

# Análise da variável Hour
hour_non_null = train_df['Hour'].notna().sum()
hour_unique = train_df['Hour'].nunique()
hour_min = train_df['Hour'].min()
hour_max = train_df['Hour'].max()

print(f"\n   Hour:")
print(f"      Observações válidas: {hour_non_null:,}")
print(f"      Valores únicos: {hour_unique:,}")
print(f"      Range: {hour_min:.2f} - {hour_max:.2f} horas")
print(f"      Interpretação: Horas desde admissão na UTI")
print(f"      Máximo: {hour_max/24:.1f} dias na UTI")

# Análise da variável HospAdmTime  
hospadm_non_null = train_df['HospAdmTime'].notna().sum()
hospadm_unique = train_df['HospAdmTime'].nunique()
hospadm_min = train_df['HospAdmTime'].min()
hospadm_max = train_df['HospAdmTime'].max()

print(f"\n   HospAdmTime:")
print(f"      Observações válidas: {hospadm_non_null:,}")
print(f"      Valores únicos: {hospadm_unique:,}")
print(f"      Range: {hospadm_min:.2f} - {hospadm_max:.2f} horas")
print(f"      Interpretação: Tempo de internação hospitalar")
print(f"      Máximo: {hospadm_max:.1f} horas para ser admitido na UTI depois de ser admitido no hospital")

# Estimativa de pacientes únicos usando características demográficas
demographic_cols = ['Age', 'Gender', 'Unit1', 'Unit2']
print(f"\nESTIMATIVA DE PACIENTES ÚNICOS:")
print(f"   (Baseada em combinação de: {', '.join(demographic_cols)})")

demo_combination = train_df[demographic_cols].fillna('Unknown')
demo_string = demo_combination.astype(str).agg('_'.join, axis=1)
unique_combinations = demo_string.nunique()

print(f"   Combinações únicas de características: {unique_combinations:,}")
print(f"   Observações por combinação (média): {len(train_df) / unique_combinations:.1f}")

combo_counts = demo_string.value_counts()
print(f"\n   DISTRIBUIÇÃO DE OBSERVAÇÕES:")
print(f"      Min observações por paciente: {combo_counts.min()}")
print(f"      Max observações por paciente: {combo_counts.max()}")
print(f"      Mediana observações por paciente: {combo_counts.median():.0f}")
print(f"      Média observações por paciente: {combo_counts.mean():.1f}")

# Padrão temporal das observações
print(f"\nPADRÃO TEMPORAL DAS OBSERVAÇÕES:")

hour_dist = train_df['Hour'].value_counts().sort_index()
print(f"   Horas com mais observações:")
top_hours = hour_dist.head(5)
for hour, count in top_hours.items():
    print(f"      Hora {hour:.0f}: {count:,} observações ({count/len(train_df)*100:.1f}%)")

unique_hours = train_df['Hour'].nunique()

print(f"   Frequência média de coleta: {len(train_df) / unique_hours:.1f} pacientes por momento")
print(f"\n   Total de momentos temporais únicos: {unique_hours:,}")

**Implicações para a análise:**

Este é um dataset de séries temporais, não de pacientes individuais, isto é, cada linha = 1 momento no tempo para 1 paciente, logo as análises devem considerar a dependência temporal, pois pacientes podem ter múltiplas observações ao longo do tempo e a variável target (SepsisLabel) pode mudar ao longo do tempo para o mesmo paciente

In [ ]:
# Visualização da estrutura temporal
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribuição das observações por hora
hour_counts = train_df['Hour'].value_counts().sort_index()
axes[0,0].plot(hour_counts.index, hour_counts.values, marker='o', linewidth=2, markersize=4)
axes[0,0].set_title('Distribuição de Observações por Hora', fontsize=12)
axes[0,0].set_xlabel('Horas desde admissão na UTI até a observação de dados clínicos')
axes[0,0].set_ylabel('Número de observações')
axes[0,0].grid(True, alpha=0.3)

# 2. Boxplot da distribuição de horas
axes[0,1].boxplot(train_df['Hour'].dropna(), patch_artist=True, 
                 boxprops=dict(facecolor='lightblue', alpha=0.7))
axes[0,1].set_title('Distribuição das Horas de Observação', fontsize=12)
axes[0,1].set_ylabel('Horas')
axes[0,1].grid(True, alpha=0.3)

# 3. Distribuição de SepsisLabel ao longo do tempo
sepsis_by_hour = train_df.groupby('Hour')['SepsisLabel'].agg(['count', 'sum']).reset_index()
sepsis_by_hour['sepsis_rate'] = sepsis_by_hour['sum'] / sepsis_by_hour['count'] * 100

# Filtrar apenas horas com pelo menos 100 observações para estabilidade
stable_hours = sepsis_by_hour[sepsis_by_hour['count'] >= 100]

axes[1,0].plot(stable_hours['Hour'], stable_hours['sepsis_rate'], 
              marker='o', linewidth=2, markersize=4, color='red')
axes[1,0].set_title('Taxa de Sepsis ao Longo do Tempo', fontsize=12)
axes[1,0].set_xlabel('Horas desde admissão na UTI até a observação de dados clínicos')
axes[1,0].set_ylabel('Taxa de Sepsis (%)')
axes[1,0].grid(True, alpha=0.3)

# 4. Histograma do número de observações por paciente estimado
demo_combination = train_df[demographic_cols].fillna('Unknown')
demo_string = demo_combination.astype(str).agg('_'.join, axis=1)
combo_counts = demo_string.value_counts()

axes[1,1].hist(combo_counts.values, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[1,1].set_title('Distribuição do Número de Observações\npor Paciente Estimado', fontsize=12)
axes[1,1].set_xlabel('Número de observações por paciente')
axes[1,1].set_ylabel('Frequência')
axes[1,1].grid(True, alpha=0.3)

# Adicionar estatísticas no gráfico
mean_obs = combo_counts.mean()
median_obs = combo_counts.median()
axes[1,1].axvline(mean_obs, color='red', linestyle='--', 
                 label=f'Média: {mean_obs:.1f}')
axes[1,1].axvline(median_obs, color='orange', linestyle='--', 
                 label=f'Mediana: {median_obs:.0f}')
axes[1,1].legend()

plt.suptitle('Estrutura Temporal do Dataset de Sepsis', fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

### Interpretação dos Gráficos da Estrutura Temporal

#### **1. Distribuição de Observações por Hora (Superior Esquerdo)**

Este gráfico revela um padrão temporal crítico no dataset:

- **Concentração inicial**: Há uma concentração massiva de observações nas primeiras horas após admissão na UTI (0-50 horas)
- **Pico máximo**: Ocorre nas primeiras horas, com mais de 30.000 observações
- **Declínio exponencial**: O número de observações diminui drasticamente após as primeiras 100 horas
- **Cauda longa**: Alguns pacientes permanecem na UTI por até 300+ horas (12+ dias)

**Implicação clínica**: A maioria dos pacientes tem estadia curta na UTI ou são transferidos/recebem alta rapidamente, enquanto uma minoria permanece por períodos prolongados.

#### **2. Distribuição das Horas de Observação (Superior Direito)**

O boxplot complementa a análise temporal:

- **Mediana baixa**: A mediana está próxima a 25-30 horas, confirmando que metade dos pacientes tem observações concentradas no início
- **Outliers extremos**: Existem casos com observações muito tardias (acima de 250 horas)
- **IQR compacto**: O intervalo interquartil é relativamente pequeno, indicando que 75% das observações ocorrem nas primeiras ~75 horas
- **Assimetria positiva**: A distribuição é fortemente enviesada para a direita

**Interpretação**: O tempo de permanência na UTI segue um padrão típico de cuidados intensivos, com a maioria dos casos sendo resolvidos rapidamente.

#### **3. Taxa de Sepsis ao Longo do Tempo (Inferior Esquerdo)**

Este é o gráfico mais clinicamente relevante, mostrando a evolução temporal do risco de sepsis:

- **Período inicial de baixo risco**: Nas primeiras 50 horas, a taxa de sepsis permanece baixa (0-5%)
- **Transição crítica**: Entre 50-100 horas, há um aumento gradual mas consistente na taxa de sepsis
- **Pico de risco**: A partir de 100 horas, a taxa de sepsis aumenta significativamente, chegando a 15-20%
- **Padrão progressivo**: O risco continua aumentando com o tempo de permanência

**Significado clínico**: 
- Pacientes com internação prolongada têm risco progressivamente maior de desenvolver sepsis
- O tempo de permanência é um **preditor temporal** importante para sepsis
- Após 100 horas na UTI, o risco praticamente triplica

#### **4. Distribuição do Número de Observações por Paciente Estimado (Inferior Direito)**

Este histograma revela a estrutura longitudinal do dataset:

- **Concentração em baixas contagens**: A maioria dos pacientes tem poucas observações (< 500)
- **Distribuição assimétrica**: Distribuição com cauda longa à direita
- **Mediana vs Média**: Mediana (36) muito menor que a média (83.2), confirmando assimetria
- **Casos extremos**: Alguns pacientes têm mais de 3.000 observações

**Interpretações importantes**:
- **Heterogeneidade temporal**: Pacientes variam drasticamente no tempo de monitoramento
- **Natureza longitudinal**: Confirma que cada paciente contribui com múltiplas observações
- **Viés potencial**: Pacientes mais graves podem ter mais observações (mais tempo na UTI)

### **Síntese dos Insights Temporais para o Relatório**

#### **Principais Descobertas:**

1. **Padrão de Permanência na UTI**:
   - 75% dos pacientes têm observações concentradas nas primeiras 75 horas
   - Mediana de permanência: ~30 horas (1,25 dias)
   - Casos extremos podem chegar a 300+ horas (12+ dias)

2. **Relação Temporal com Sepsis**:
   - **Janela de baixo risco**: Primeiras 50 horas (taxa < 5%)
   - **Janela de transição**: 50-100 horas (aumento gradual)
   - **Janela de alto risco**: >100 horas (taxa pode ultrapassar 15%)

3. **Estrutura dos Dados**:
   - Dataset é uma **coleção de séries temporais**, não registros únicos de pacientes
   - Cada linha = um momento temporal para um paciente específico
   - Variabilidade alta no número de observações por paciente (mediana: 36, máximo: >3000)

#### **Implicações para Modelagem**:

- **Dependência temporal**: Modelos devem considerar a sequência temporal das observações
- **Feature temporal**: O tempo de permanência é um preditor importante
- **Validação temporal**: Necessária validação que preserve a ordem temporal
- **Threshold temporal**: Considerar diferentes estratégias para janelas temporais específicas

#### **Relevância Clínica**:

- O tempo de permanência prolongado na UTI está associado ao aumento progressivo do risco de sepsis
- Pacientes que permanecem >100 horas na UTI constituem um grupo de alto risco que requer monitoramento intensificado
- A detecção precoce de sepsis é mais crítica nos primeiros dias de internação, quando a taxa é naturalmente baixa

### 2.4 Separação de Features e Target

Separamos a variável alvo (SepsisLabel) das features para facilitar as análises subsequentes. Essa separação é fundamental para a análise univariada e bivariada posterior.

In [ ]:
# Separação entre features (X) e variável target (y)
X_train = train_df.drop('SepsisLabel', axis=1)
y_train = train_df['SepsisLabel']

print("SEPARAÇÃO DE FEATURES E TARGET")
print("=" * 40)
print(f"Shape das features (X_train): {X_train.shape}")
print(f"Shape do target (y_train): {y_train.shape}")

# Verificar se há duplicatas no dataset
duplicates = train_df.duplicated().sum()
print(f"\nLinhas duplicadas: {duplicates}")

# Análise de valores únicos por coluna
print(f"\nANÁLISE DE VALORES ÚNICOS POR COLUNA")
print("=" * 50)
unique_counts = train_df.nunique().sort_values(ascending=False)
print("Valores únicos por coluna:")
display(unique_counts.to_frame('Valores_Únicos'))

print(f"\nColunas com poucos valores únicos (potencialmente categóricas):")
few_unique = unique_counts[unique_counts <= 10]
display(few_unique.to_frame('Valores_Únicos'))

# Análise da distribuição da variável target
print(f"\nDISTRIBUIÇÃO DA VARIÁVEL TARGET")
print("=" * 50)
target_counts = y_train.value_counts().sort_index()
target_pct = y_train.value_counts(normalize=True).sort_index() * 100

for label, count in target_counts.items():
    pct = target_pct[label]
    status = "Não-Sepsis" if label == 0 else "Sepsis"
    print(f"{status} ({label}): {count:,} amostras ({pct:.2f}%)")

# Calcular razão de desbalanceamento
imbalance_ratio = target_counts[0] / target_counts[1]
print(f"\nRazão de desbalanceamento: {imbalance_ratio:.1f}:1")
print(f"Para cada caso de sepsis, existem {imbalance_ratio:.1f} casos de não-sepsis")

#### 2.4.1 Visualização da Distribuição da Variável Target

In [ ]:
# Visualização da distribuição da variável target
plt.figure(figsize=(15, 5))

# Gráfico 1: Contagem absoluta
plt.subplot(1, 2, 1)
bars = plt.bar(['Não-Sepsis', 'Sepsis'], target_counts.values, 
               color=['#3498db', '#e74c3c'], alpha=0.8, edgecolor='black')
plt.title('Distribuição Absoluta das Classes', fontsize=14, pad=20)
plt.xlabel('Classes')
plt.ylabel('Frequência')

# Adicionar valores nas barras
for bar, count in zip(bars, target_counts.values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(target_counts)*0.01,
            f'{count:,}', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Gráfico de pizza
plt.subplot(1, 2, 2)
wedges, texts, autotexts = plt.pie(target_pct.values, 
                                  labels=['Não-Sepsis', 'Sepsis'], 
                                  autopct='%1.2f%%', 
                                  colors=['#3498db', '#e74c3c'],
                                  startangle=90,
                                  explode=(0, 0.1))
plt.title('Proporção das Classes', fontsize=14, pad=20)

# Melhorar formatação dos rótulos
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.show()

Podemos ver que o dataset está altamente desbalanceado. Precisaremos tomar alguns cuidados na fase de avaliação e amostragem:
- Utilizar métricas adequadas (F1-Score, AUC-ROC, Precision-Recall)
- Aplicar técnicas de balanceamento (SMOTE, undersampling)
- Configurar class_weight nos algoritmos de ML
- Considerar threshold optimization

## Análise Univariada Detalhada

A análise univariada examina cada variável individualmente para compreender suas distribuições, identificar outliers e avaliar características estatísticas importantes como assimetria e curtose.

In [ ]:
# 2. Histogramas com KDE para análise de distribuição das variáveis numéricas

# Selecionar variáveis numéricas com baixo percentual de missing values
numerical_columns = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Filtrar colunas com menos de 30% de valores faltantes para análise
low_missing_cols = []
for col in numerical_columns:
    missing_pct = X_train[col].isnull().sum() / len(X_train) * 100
    if missing_pct < 30:
        low_missing_cols.append(col)

print(f"Analisando {len(low_missing_cols)} variáveis numéricas com <30% missing values")

# Selecionar as primeiras 12 variáveis para visualização
cols_to_plot = low_missing_cols[:12]

n_cols = 3
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(18, n_rows * 4))

for idx, feature in enumerate(cols_to_plot, 1):
    plt.subplot(n_rows, n_cols, idx)
    
    # Calcular skewness (assimetria)
    skewness = X_train[feature].skew()
    
    # Plotar histograma com KDE
    sns.histplot(X_train[feature].dropna(), kde=True, alpha=0.7, color='skyblue')
    plt.title(f"{feature} | Skewness: {skewness:.2f}", fontsize=12)
    plt.xlabel(feature)
    plt.ylabel('Frequência')
    
    # Adicionar linha vertical para a média
    mean_val = X_train[feature].mean()
    plt.axvline(mean_val, color='red', linestyle='--', alpha=0.7, label=f'Média: {mean_val:.2f}')
    plt.legend()

plt.tight_layout()
plt.show()

print(f"\nResumo das {len(cols_to_plot)} variáveis analisadas:")
for col in cols_to_plot:
    skew_val = X_train[col].skew()
    if skew_val > 1:
        interpretation = "Fortemente assimétrica à direita"
    elif skew_val < -1:
        interpretation = "Fortemente assimétrica à esquerda"
    elif abs(skew_val) < 0.5:
        interpretation = "Aproximadamente simétrica"
    else:
        interpretation = "Moderadamente assimétrica"
    
    print(f"  {col}: {skew_val:.2f} ({interpretation})")


Interpretação dos gráficos de assimetria:
- Skewness = 0: Distribuição simétrica
- Skewness > 1: Fortemente assimétrica à direita
- Skewness < -1: Fortemente assimétrica à esquerda
- -1 ≤ Skewness ≤ 1: Moderadamente assimétrica ou simétrica

In [ ]:
# 3. Swarm Plot para identificar outliers (Não rodar essa célula: muito exemplos para rodar de uma vez)
# Obs: usar amostragem para conseguir usar o swarm plot
# Análise de outliers em variáveis importantes vs target

# Selecionar algumas variáveis importantes para análise de outliers
important_vars = ['HR', 'SBP', 'DBP', 'Temp', 'Resp', 'O2Sat', 'Age']
available_vars = [col for col in important_vars if col in X_train.columns]

# Criar subplots para swarm plots
n_vars = min(4, len(available_vars))  # Limitar a 4 para não sobrecarregar
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, var in enumerate(available_vars[:n_vars]):
    # Preparar dados removendo NaN
    temp_df = pd.DataFrame({
        'variable': X_train[var],
        'target': y_train
    }).dropna()
    
    # Converter target para string para melhor visualização
    temp_df['target_str'] = temp_df['target'].map({0: 'Não-Sepsis', 1: 'Sepsis'})
    
    if len(temp_df) > 0:
        # Criar swarm plot
        sns.swarmplot(data=temp_df, x='target_str', y='variable', 
                        palette=['#3498db', '#e74c3c'], alpha=0.6, ax=axes[idx])
        axes[idx].set_title(f'Swarm Plot: {var} vs SepsisLabel', fontsize=12)
        axes[idx].set_xlabel('SepsisLabel')
        axes[idx].set_ylabel(var)
        
        # Adicionar informações estatísticas
        stats_0 = temp_df[temp_df['target'] == 0]['variable'].describe()
        stats_1 = temp_df[temp_df['target'] == 1]['variable'].describe()
        
        info_text = f"Não-Sepsis: μ={stats_0['mean']:.2f}\\nSepsis: μ={stats_1['mean']:.2f}"
        axes[idx].text(0.02, 0.98, info_text, transform=axes[idx].transAxes,
                        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                        fontsize=9)
    else:
        axes[idx].text(0.5, 0.5, f'Dados insuficientes\\npara {var}', 
                        transform=axes[idx].transAxes, ha='center', va='center')
        axes[idx].set_title(f'{var} - Sem dados')

# Remover subplots vazios
for i in range(n_vars, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

print("\\nINTERPRETAÇÃO DOS SWARM PLOTS:")
print("=" * 50)
print("- Pontos isolados e distantes dos clusters principais = outliers")
print("- Densidade maior de pontos = concentração de valores")
print("- Diferentes posições verticais entre classes = possível discriminação")
print("- Sobreposição entre classes = dificuldade de separação")


## Análise Bivariada

A análise bivariada examina as relações entre duas variáveis, ajudando a identificar padrões, dependências e correlações que podem ser cruciais para o desenvolvimento de modelos preditivos.

In [ ]:
# 1. Pair Plot para mostrar distribuições e relações entre variáveis
# Selecionar um subconjunto de variáveis para o pairplot (para performance)

# Pegar variáveis com baixo missing e adicionar o target
pairplot_vars = []
if len(low_missing_cols) > 0:
    # Selecionar as 5 primeiras variáveis numéricas com baixo missing
    pairplot_vars = low_missing_cols[:5]

# Criar DataFrame para pairplot incluindo o target
if len(pairplot_vars) > 0:
    pairplot_data = X_train[pairplot_vars].copy()
    pairplot_data['SepsisLabel'] = y_train
    
    # Amostrar dados se o dataset for muito grande (para performance)
    if len(pairplot_data) > 5000:
        pairplot_sample = pairplot_data.sample(n=5000, random_state=42)
        print(f"Usando amostra de 5000 registros para pairplot (de {len(pairplot_data)} total)")
    else:
        pairplot_sample = pairplot_data
        print(f"Usando todos os {len(pairplot_data)} registros para pairplot")
    
    # Remover linhas com valores faltantes para o pairplot
    pairplot_sample = pairplot_sample.dropna()
    
    if len(pairplot_sample) > 0:
        print(f"\\nCriando Pair Plot com {len(pairplot_sample)} amostras válidas...")
        print(f"Variáveis incluídas: {pairplot_vars}")
        
        # Configurar paleta de cores
        sns.set_palette("Set1")
        
        # Criar pairplot
        plt.figure(figsize=(15, 12))
        
        # Pairplot com diferenciação por classe
        pair_grid = sns.pairplot(pairplot_sample, hue='SepsisLabel', 
                                diag_kind='hist', 
                                plot_kws={'alpha': 0.6},
                                diag_kws={'alpha': 0.7})
        
        # Personalizar a legenda
        pair_grid._legend.set_title("SepsisLabel")
        for text, label in zip(pair_grid._legend.get_texts(), ['Não-Sepsis', 'Sepsis']):
            text.set_text(label)
        
        plt.suptitle('Pair Plot - Análise de Relações entre Variáveis', y=1.02, fontsize=16)
        plt.show()
        
        print("\\nINTERPRETAÇÃO DO PAIR PLOT:")
        print("=" * 50)
        print("- Diagonal: Histogramas mostram a distribuição individual de cada variável")
        print("- Triângulo inferior/superior: Scatter plots mostram relações entre pares de variáveis")
        print("- Cores diferentes: Separação por classe (Sepsis vs Não-Sepsis)")
        print("- Padrões lineares nos scatter plots: Possíveis correlações")
        print("- Separação clara entre cores: Potencial discriminativo da variável")
        
    else:
        print("Dados insuficientes após remoção de valores faltantes para pairplot.")
        
else:
    print("Nenhuma variável numérica disponível para pairplot.")

In [ ]:
# 2. Box Plots para análise de distribuições por classe
# Examinar como variáveis numéricas se comportam para cada classe do target

# Selecionar variáveis para box plots
boxplot_vars = available_vars[:4] if available_vars else low_missing_cols[:4]

if len(boxplot_vars) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    for idx, var in enumerate(boxplot_vars):
        # Preparar dados
        temp_df = pd.DataFrame({
            'variable': X_train[var],
            'target': y_train
        }).dropna()
        
        # Converter target para labels descritivos
        temp_df['target_str'] = temp_df['target'].map({0: 'Não-Sepsis', 1: 'Sepsis'})
        
        if len(temp_df) > 0:
            # Criar box plot
            sns.boxplot(data=temp_df, x='target_str', y='variable', 
                       palette=['#3498db', '#e74c3c'], ax=axes[idx])
            axes[idx].set_title(f'Box Plot: {var} vs SepsisLabel', fontsize=12)
            axes[idx].set_xlabel('SepsisLabel')
            axes[idx].set_ylabel(var)
            
            # Calcular e mostrar estatísticas
            stats_by_class = temp_df.groupby('target')['variable'].agg(['median', 'mean', 'std']).round(2)
            
            # Adicionar informações estatísticas
            info_text = f"Não-Sepsis - Med: {stats_by_class.loc[0, 'median']:.2f}\\n"
            info_text += f"Sepsis - Med: {stats_by_class.loc[1, 'median']:.2f}"
            
            axes[idx].text(0.02, 0.98, info_text, transform=axes[idx].transAxes,
                          verticalalignment='top', 
                          bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                          fontsize=10)
        else:
            axes[idx].text(0.5, 0.5, f'Dados insuficientes\\npara {var}', 
                          transform=axes[idx].transAxes, ha='center', va='center')
            axes[idx].set_title(f'{var} - Sem dados')
    
    plt.tight_layout()
    plt.show()
    
    # Análise estatística das diferenças
    print("\\nANÁLISE ESTATÍSTICA DAS DIFERENÇAS:")
    print("=" * 50)
    
    for var in boxplot_vars:
        if var in X_train.columns:
            temp_df = pd.DataFrame({
                'variable': X_train[var],
                'target': y_train
            }).dropna()
            
            if len(temp_df) > 10:  # Mínimo de dados para análise
                group_0 = temp_df[temp_df['target'] == 0]['variable']
                group_1 = temp_df[temp_df['target'] == 1]['variable']
                
                if len(group_0) > 0 and len(group_1) > 0:
                    median_diff = group_1.median() - group_0.median()
                    mean_diff = group_1.mean() - group_0.mean()
                    
                    print(f"{var}:")
                    print(f"  Diferença de medianas (Sepsis - Não-Sepsis): {median_diff:.3f}")
                    print(f"  Diferença de médias (Sepsis - Não-Sepsis): {mean_diff:.3f}")
    
else:
    print("Nenhuma variável disponível para box plots.")

INTERPRETAÇÃO DOS BOX PLOTS
- Caixa (box): Representa o IQR (Intervalo Interquartil)
- Linha central: Mediana da distribuição
- Whiskers (bigodes): Extensão até valores dentro de 1.5×IQR
- Pontos individuais: Outliers (valores extremos)
- Caixas mais longas: Maior variabilidade nos dados
- Diferenças entre medianas das classes: Potencial discriminativo

## Análise Multivariada

A análise multivariada examina as interações entre múltiplas variáveis simultaneamente, identificando padrões complexos e correlações que podem não ser aparentes na análise univariada ou bivariada.

In [ ]:
# Correlation Heatmap - Análise de correlações entre variáveis

# Selecionar variáveis numéricas com baixo missing para análise de correlação
correlation_vars = low_missing_cols[:20] if len(low_missing_cols) > 20 else low_missing_cols

if len(correlation_vars) > 1:
    # Criar subset dos dados para correlação
    correlation_data = X_train[correlation_vars].copy()
    
    # Calcular matriz de correlação
    correlation_matrix = correlation_data.corr()
    
    print(f"MATRIZ DE CORRELAÇÃO")
    print("=" * 50)
    print(f"Analisando correlações entre {len(correlation_vars)} variáveis numéricas")
    print(f"Variáveis incluídas: {correlation_vars[:10]}{'...' if len(correlation_vars) > 10 else ''}")
    
    # Criar heatmap de correlação
    plt.figure(figsize=(16, 14))
    
    # Configurar máscara para a diagonal superior (opcional)
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    
    # Criar heatmap
    sns.heatmap(correlation_matrix, 
                annot=True,           # Mostrar valores
                fmt='.2f',           # Formato dos números
                cmap='RdBu_r',       # Paleta de cores (similar ao GeeksforGeeks)
                center=0,            # Centralizar em zero
                square=True,         # Células quadradas
                linewidths=0.5,      # Linhas entre células
                cbar_kws={"shrink": .8},  # Configurar colorbar
                mask=mask)           # Máscara para diagonal superior
    
    plt.title('Matriz de Correlação - Heatmap', fontsize=16, pad=20)
    plt.xlabel('Variáveis', fontsize=12)
    plt.ylabel('Variáveis', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Análise de correlações mais significativas
    print("\\nANÁLISE DE CORRELAÇÕES SIGNIFICATIVAS:")
    print("=" * 50)
    
    # Encontrar correlações mais altas (excluindo diagonal)
    correlation_matrix_no_diag = correlation_matrix.copy()
    np.fill_diagonal(correlation_matrix_no_diag.values, np.nan)
    
    # Achatar a matriz e ordenar por valor absoluto
    corr_pairs = []
    for i in range(len(correlation_matrix_no_diag)):
        for j in range(i+1, len(correlation_matrix_no_diag)):
            var1 = correlation_matrix_no_diag.index[i]
            var2 = correlation_matrix_no_diag.columns[j]
            corr_value = correlation_matrix_no_diag.iloc[i, j]
            if not np.isnan(corr_value):
                corr_pairs.append((var1, var2, corr_value))
    
    # Ordenar por valor absoluto de correlação
    corr_pairs_sorted = sorted(corr_pairs, key=lambda x: abs(x[2]), reverse=True)
    
    print("Top 10 correlações mais fortes:")
    for i, (var1, var2, corr) in enumerate(corr_pairs_sorted[:10], 1):
        strength = "Muito forte" if abs(corr) > 0.8 else "Forte" if abs(corr) > 0.6 else "Moderada" if abs(corr) > 0.4 else "Fraca"
        direction = "positiva" if corr > 0 else "negativa"
        print(f"{i:2d}. {var1} ↔ {var2}: {corr:.3f} ({strength} {direction})")
    
    # Identificar correlações problemáticas (multicolinearidade)
    high_corr_pairs = [pair for pair in corr_pairs_sorted if abs(pair[2]) > 0.8]
    if high_corr_pairs:
        print(f"Atenção: {len(high_corr_pairs)} pares com correlação muito alta (>0.8) detectados!")
        print("Considere técnicas para lidar com multicolinearidade:")
        print("- Remoção de uma das variáveis correlacionadas")
        print("- Análise de Componentes Principais (PCA)")
        print("- Regularização (Ridge, Lasso)")
    
    print("\\nINTERPRETAÇÃO DA MATRIZ DE CORRELAÇÃO:")
    print("=" * 50)
    print("- Valores próximos a +1: Correlação positiva forte (variáveis aumentam juntas)")
    print("- Valores próximos a -1: Correlação negativa forte (uma aumenta, outra diminui)")
    print("- Valores próximos a 0: Sem correlação linear")
    print("- Cores mais escuras (vermelho/azul): Correlações mais fortes")
    print("- Cores mais claras: Correlações mais fracas")
    
else:
    print("Dados insuficientes para análise de correlação.")

## Análise das Variáveis Categóricas

A identificação e análise das variáveis categóricas nos permite compreender as características demográficas e clínicas dos pacientes, bem como sua associação com o desenvolvimento de sepsis.

In [ ]:
# Identificação de variáveis categóricas
print("IDENTIFICAÇÃO DE VARIÁVEIS CATEGÓRICAS")
print("=" * 50)

# Variáveis categóricas explícitas
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Variáveis numéricas que podem ser categóricas (poucos valores únicos)
potential_categorical = []
for col in X_train.columns:
    unique_values = X_train[col].nunique()
    if unique_values <= 10 and X_train[col].dtype in ['int64', 'float64']:
        potential_categorical.append((col, unique_values))

print(f"Variáveis categóricas explícitas: {len(categorical_cols)}")
if categorical_cols:
    print(f"  {categorical_cols}")

print(f"\nVariáveis potencialmente categóricas (≤10 valores únicos): {len(potential_categorical)}")
for col, count in potential_categorical[:10]:
    print(f"  {col}: {count} valores únicos")

# Selecionar variáveis categóricas relevantes para análise
important_categorical = ['Gender', 'Unit1', 'Unit2']
selected_categorical = []

for col in important_categorical:
    if col in X_train.columns:
        selected_categorical.append(col)

# Adicionar outras variáveis categóricas se necessário
for col, count in potential_categorical:
    if col not in selected_categorical and len(selected_categorical) < 5:
        selected_categorical.append(col)

print(f"\nVariáveis selecionadas para análise detalhada: {selected_categorical}")

## Análise Detalhada das Variáveis Categóricas Selecionadas

Vamos examinar estatisticamente cada variável categórica selecionada, incluindo sua distribuição e relação com a variável target.

In [ ]:
# Análise estatística detalhada das variáveis categóricas selecionadas
for col in selected_categorical:
    print(f"\nANÁLISE DA VARIÁVEL: {col}")
    print("=" * 60)
    
    # Distribuição de frequências
    value_counts = X_train[col].value_counts()
    value_pct = X_train[col].value_counts(normalize=True) * 100
    
    print("Distribuição de frequências:")
    for value in value_counts.index:
        count = value_counts[value]
        pct = value_pct[value]
        print(f"  {value}: {count:,} ({pct:.1f}%)")
    
    print(f"\nEstatísticas básicas:")
    print(f"  Valores únicos: {X_train[col].nunique()}")
    print(f"  Valores faltantes: {X_train[col].isnull().sum()}")
    print(f"  Moda: {X_train[col].mode().iloc[0] if len(X_train[col].mode()) > 0 else 'N/A'}")
    
    # Análise da relação com a variável target
    if len(value_counts) <= 10:  # Só para variáveis com poucos valores
        print(f"\nRelação com SepsisLabel:")
        
        # Tabela de contingência
        crosstab = pd.crosstab(X_train[col], y_train, margins=True)
        print("Tabela de contingência:")
        display(crosstab)
        
        # Proporções condicionais
        crosstab_pct = pd.crosstab(X_train[col], y_train, normalize='index') * 100
        print(f"\nTaxa de sepsis por categoria:")
        for category in crosstab_pct.index:
            if category != 'All':
                sepsis_rate = crosstab_pct.loc[category, 1] if 1 in crosstab_pct.columns else 0
                print(f"  {category}: {sepsis_rate:.2f}% de casos com sepsis")
    
    print("-" * 60)

## Visualização das Variáveis Categóricas

As visualizações nos permitem compreender melhor a distribuição das variáveis categóricas e sua associação com a ocorrência de sepsis de forma intuitiva.

In [ ]:
# Visualização das variáveis categóricas
if selected_categorical:
    n_vars = len(selected_categorical)
    fig, axes = plt.subplots(n_vars, 2, figsize=(16, 6*n_vars))
    
    if n_vars == 1:
        axes = axes.reshape(1, -1)
    
    for i, col in enumerate(selected_categorical):
        # Gráfico 1: Distribuição simples
        counts = X_train[col].value_counts()
        
        ax1 = axes[i, 0] if n_vars > 1 else axes[0]
        bars = ax1.bar(range(len(counts)), counts.values, 
                      color='#2c3e50', alpha=0.8, edgecolor='black')
        ax1.set_xticks(range(len(counts)))
        ax1.set_xticklabels(counts.index, rotation=45)
        ax1.set_title(f'Distribuição de {col}')
        ax1.set_xlabel(col)
        ax1.set_ylabel('Frequência')
        
        # Adicionar valores nas barras
        for bar, count in zip(bars, counts.values):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(counts)*0.01,
                    f'{count:,}', ha='center', va='bottom', fontsize=9)
        
        # Gráfico 2: Relação com target
        ax2 = axes[i, 1] if n_vars > 1 else axes[1]
        
        # Criar tabela de contingência para visualização
        temp_df = pd.DataFrame({'categoria': X_train[col], 'target': y_train})
        crosstab = pd.crosstab(temp_df['categoria'], temp_df['target'])
        crosstab_pct = pd.crosstab(temp_df['categoria'], temp_df['target'], normalize='index') * 100
        
        # Gráfico de barras empilhadas
        crosstab.plot(kind='bar', stacked=True, ax=ax2, 
                     color=['#3498db', '#e74c3c'], alpha=0.8)
        ax2.set_title(f'{col} vs SepsisLabel')
        ax2.set_xlabel(col)
        ax2.set_ylabel('Frequência')
        ax2.legend(['Não-Sepsis', 'Sepsis'], title='SepsisLabel')
        ax2.tick_params(axis='x', rotation=45)
        
        # Adicionar percentuais de sepsis
        for j, category in enumerate(crosstab.index):
            if 1 in crosstab_pct.columns:
                sepsis_pct = crosstab_pct.loc[category, 1]
                total_height = crosstab.loc[category].sum()
                ax2.text(j, total_height + max(crosstab.sum(axis=1))*0.02, 
                        f'{sepsis_pct:.1f}%', ha='center', va='bottom', 
                        fontweight='bold', fontsize=8, color='red')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Nenhuma variável categórica disponível para visualização.")

## Análise de Relações entre Variáveis Categóricas e Numéricas

Esta análise explora como as variáveis categóricas influenciam a distribuição das variáveis numéricas, especialmente no contexto da sepsis. Isso nos ajuda a identificar padrões clínicos importantes.

In [ ]:
# Seleção de variáveis numéricas relevantes para análise
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Filtrar variáveis numéricas com menos de 50% de valores faltantes
numeric_cols_clean = []
for col in numeric_cols:
    missing_pct = X_train[col].isnull().sum() / len(X_train) * 100
    if missing_pct < 50:
        numeric_cols_clean.append(col)

print("SELEÇÃO DE VARIÁVEIS NUMÉRICAS")
print("=" * 50)
print(f"Total de variáveis numéricas: {len(numeric_cols)}")
print(f"Variáveis com <50% missing: {len(numeric_cols_clean)}")

# Priorizar variáveis de sinais vitais importantes
important_numeric = ['HR', 'SBP', 'DBP', 'Temp', 'Resp', 'O2Sat', 'Age']
selected_numeric = []

for col in important_numeric:
    if col in numeric_cols_clean:
        selected_numeric.append(col)
    if len(selected_numeric) >= 3:
        break

# Completar com outras variáveis se necessário
if len(selected_numeric) < 3:
    for col in numeric_cols_clean:
        if col not in selected_numeric and len(selected_numeric) < 3:
            selected_numeric.append(col)

print(f"Variáveis numéricas selecionadas: {selected_numeric}")
print(f"Variáveis categóricas disponíveis: {selected_categorical}")

## Análise Estatística das Relações Categórica vs Numérica

Vamos examinar estatisticamente como as variáveis categóricas influenciam as distribuições das variáveis numéricas.

In [ ]:
# Análise estatística das relações entre variáveis categóricas e numéricas
for cat_col in selected_categorical[:2]:  # Limitar a 2 categóricas para não sobrecarregar
    for num_col in selected_numeric[:2]:  # Limitar a 2 numéricas por categórica
        
        print(f"\nANÁLISE: {num_col} por {cat_col}")
        print("=" * 60)
        
        # Preparar dados removendo valores faltantes
        temp_df = pd.DataFrame({
            'categorical': X_train[cat_col],
            'numerical': X_train[num_col],
            'target': y_train
        }).dropna()
        
        if len(temp_df) > 0:
            print(f"Amostras válidas para análise: {len(temp_df):,}")
            
            # Estatísticas descritivas por categoria
            stats_by_cat = temp_df.groupby('categorical')['numerical'].agg([
                'count', 'mean', 'std', 'min', 'max', 'median'
            ]).round(2)
            
            print(f"\nEstatísticas descritivas de {num_col} por {cat_col}:")
            display(stats_by_cat)
            
            # Estatísticas por categoria E target (sepsis)
            stats_by_cat_target = temp_df.groupby(['categorical', 'target'])['numerical'].agg([
                'count', 'mean', 'std'
            ]).round(2)
            
            print(f"\nEstatísticas de {num_col} por {cat_col} e SepsisLabel:")
            display(stats_by_cat_target)
            
            # Teste estatístico para diferença de médias (se aplicável)
            categories = temp_df['categorical'].unique()
            if len(categories) == 2:
                from scipy import stats
                
                group1 = temp_df[temp_df['categorical'] == categories[0]]['numerical']
                group2 = temp_df[temp_df['categorical'] == categories[1]]['numerical']
                
                if len(group1) > 1 and len(group2) > 1:
                    # Teste t para amostras independentes
                    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)
                    
                    print(f"\nTeste t para diferença de médias entre categorias:")
                    print(f"Estatística t: {t_stat:.4f}")
                    print(f"P-valor: {p_value:.4f}")
                    
                    alpha = 0.05
                    if p_value < alpha:
                        print(f"Resultado: Diferença estatisticamente significativa (p < {alpha})")
                    else:
                        print(f"Resultado: Diferença não estatisticamente significativa (p >= {alpha})")
            
        else:
            print("Dados insuficientes após remoção de valores faltantes.")
        
        print("-" * 60)

## Visualização das Relações Categórica vs Numérica

Os boxplots nos permitem visualizar como as distribuições das variáveis numéricas variam entre as diferentes categorias, incluindo a presença de sepsis.

In [ ]:
# Visualização das relações entre variáveis categóricas e numéricas
if selected_categorical and selected_numeric:
    # Calcular número de combinações para subplot
    n_combinations = min(4, len(selected_categorical) * len(selected_numeric))
    
    if n_combinations > 0:
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        combination_count = 0
        
        for cat_col in selected_categorical[:2]:  # Máximo 2 categóricas
            for num_col in selected_numeric[:2]:  # 2 numéricas por categórica
                if combination_count >= n_combinations:
                    break
                
                # Preparar dados removendo valores faltantes
                temp_df = pd.DataFrame({
                    'categorical': X_train[cat_col],
                    'numerical': X_train[num_col],
                    'target': y_train
                }).dropna()
                
                if len(temp_df) > 0:
                    ax = axes[combination_count]
                    
                    # Criar boxplot com separação por target
                    sns.boxplot(data=temp_df, x='categorical', y='numerical', 
                               hue='target', ax=ax, palette=['#3498db', '#e74c3c'])
                    
                    ax.set_title(f'{num_col} por {cat_col} e SepsisLabel', fontsize=12, pad=15)
                    ax.set_xlabel(cat_col)
                    ax.set_ylabel(num_col)
                    ax.tick_params(axis='x', rotation=45)
                    
                    # Configurar legenda
                    handles, labels = ax.get_legend_handles_labels()
                    ax.legend(handles, ['Não-Sepsis', 'Sepsis'], title='SepsisLabel')
                    
                    # Adicionar informação sobre o tamanho da amostra
                    n_points = len(temp_df)
                    ax.text(0.02, 0.98, f'n = {n_points:,}', transform=ax.transAxes, 
                           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                
                combination_count += 1
        
        # Remover subplots vazios
        for i in range(combination_count, len(axes)):
            fig.delaxes(axes[i])
        
        plt.tight_layout()
        plt.show()
    
    else:
        print("Combinações insuficientes para visualização.")
        
else:
    print("Variáveis insuficientes para análise de relações.")

## Conclusões da Análise Exploratória

Vamos consolidar os principais insights obtidos durante nossa análise exploratória e suas implicações para o desenvolvimento de modelos preditivos.

In [ ]:
# Consolidação dos principais insights da análise exploratória
print("RESUMO EXECUTIVO DA ANÁLISE EXPLORATÓRIA")
print("=" * 60)

print(f"\n1. ESTRUTURA DO DATASET")
print("-" * 30)
print(f"   • Dimensões: {train_df.shape[0]:,} amostras x {train_df.shape[1]} features")
print(f"   • Tamanho em memória: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   • Duplicatas: {train_df.duplicated().sum()} linhas")

print(f"\n2. DISTRIBUIÇÃO DO TARGET")
print("-" * 30)
print(f"   • Não-Sepsis: {target_counts[0]:,} ({target_pct[0]:.2f}%)")
print(f"   • Sepsis: {target_counts[1]:,} ({target_pct[1]:.2f}%)")
print(f"   • Razão de desbalanceamento: {imbalance_ratio:.1f}:1")
print(f"   • Classificação: {'Altamente desbalanceado' if imbalance_ratio > 10 else 'Moderadamente desbalanceado'}")

print(f"\n3. QUALIDADE DOS DADOS")
print("-" * 30)
if len(missing_with_nans) > 0:
    print(f"   • Colunas com missing values: {len(missing_with_nans)}/{len(X_train.columns)}")
    print(f"   • Total de valores faltantes: {missing_count.sum():,}")
    print(f"   • Porcentagem do dataset: {(missing_count.sum() / (len(X_train) * len(X_train.columns)))*100:.2f}%")
    
    # Categorização dos missing values
    low_missing = missing_with_nans[missing_with_nans['Porcentagem'] < 20]
    medium_missing = missing_with_nans[(missing_with_nans['Porcentagem'] >= 20) & 
                                      (missing_with_nans['Porcentagem'] < 80)]
    high_missing = missing_with_nans[missing_with_nans['Porcentagem'] >= 80]
    
    print(f"   • Baixo missing (<20%): {len(low_missing)} colunas")
    print(f"   • Médio missing (20-80%): {len(medium_missing)} colunas")
    print(f"   • Alto missing (≥80%): {len(high_missing)} colunas")
else:
    print(f"   • Dataset sem valores faltantes")

print(f"\n4. ANÁLISE UNIVARIADA")
print("-" * 30)
print(f"   • Variáveis numéricas analisadas: {len(low_missing_cols) if 'low_missing_cols' in locals() else 'N/A'}")
print(f"   • Distribuição do target: Altamente desbalanceada")
print(f"   • Assimetria identificada em múltiplas variáveis")
print(f"   • Outliers detectados em várias features")

print(f"\n5. ANÁLISE BIVARIADA")
print("-" * 30)
print(f"   • Pair plots revelaram relações entre variáveis")
print(f"   • Box plots mostraram diferenças de distribuição entre classes")
print(f"   • Potencial discriminativo identificado em múltiplas features")

print(f"\n6. ANÁLISE MULTIVARIADA")
print("-" * 30)
if 'correlation_matrix' in locals():
    high_corr_count = len([pair for pair in corr_pairs_sorted if abs(pair[2]) > 0.8]) if 'corr_pairs_sorted' in locals() else 0
    print(f"   • Matriz de correlação gerada com {len(correlation_vars)} variáveis")
    print(f"   • Correlações muito altas (>0.8): {high_corr_count} pares")
    print(f"   • Potencial multicolinearidade {'detectada' if high_corr_count > 0 else 'não detectada'}")
else:
    print(f"   • Análise de correlação executada")

print(f"\n7. VARIÁVEIS CATEGÓRICAS")
print("-" * 30)
print(f"   • Variáveis identificadas: {len(selected_categorical) if 'selected_categorical' in locals() else 'N/A'}")
print(f"   • Variáveis analisadas: {selected_categorical if 'selected_categorical' in locals() else 'N/A'}")

print(f"\n8. RECOMENDAÇÕES PARA MODELAGEM")
print("-" * 30)
print(f"   • Tratar desbalanceamento de classes (SMOTE, class_weight)")
print(f"   • Estratégia de imputação para missing values")
print(f"   • Normalização/padronização das features")
print(f"   • Feature selection para reduzir dimensionalidade")
print(f"   • Validação cruzada estratificada")
print(f"   • Métricas adequadas: AUC-ROC, F1-Score, Precision-Recall")
